# **Exercises 7:** Machine learning

In this exercise sheet, we will study and train neural networks (NN). We are going to apply some common machine
learning methods to a particle physics task: the separation of jets originating from top quarks from jets originating
from gluons in events recorded by the ATLAS detector.

This exercise was adapted from a [previous exercise](https://github.com/dkgithub/wuhan_DL_labs/tree/master/top-tagging)
with courtesy from Lisa Benato. The data used in this exercise was provided by
[Gregor Kasieczka et al](https://arxiv.org/abs/1902.09914).

Work on that exercise sheet by using the [UHH Jupyter server](https://code.min.uni-hamburg.de/). Login
with your UHH credentials (your account with username `bXXXX`).
The default image includes all packages that are required for executing the code. This notebook has been tested with
*Python 3.11.2* and *Tensorflow 2.18*.

The code for that exercise sheet as well as the instructions have been prepared in a *Jupyter* notebook. You can 
retrieve the exercise material (notebook, data and exercise sheet) using `git`. For that follow these steps:

- If you haven't done yet clone the repository. Login to the *UHH Jupyter Machine*. Go to the folder where you want
  to place the new repository. Select *Git > Clone a Repository* in the toolbar on the top of the *Jupyter Lab* web
  interface. Now a dialog that asks for a web address should open. Enter the URL
  [https://github.com/pkausw/ml_exercise.git](https://github.com/pkausw/ml_exercise.git) of the exercise.

  After having pressed the *Clone* button the download of the repository starts. A new folder with the name of
  the repository should appear in the file browser sidebar.

- To update the repository contents go to the repository folder that you have cloned. For retrieving the recent version
  of the repository go to *Git > Pull from Remote* in the toolbar on the top of the *Jupyter Lab* web interface.

- Next, copy the input data for this exercise to the machine. For this, open a new terminal by clicking on the *+* sign at the top of this window and select *Other > Terminal*. Execute the following command: 
`wget https://syncandshare.desy.de/index.php/s/rKrtHqbQwb5TAfg/download -O dataset.zip`. This will create the file `dataset.zip` in your current working directory.

- Unpack this new file with `unzip dataset.zip`. You will now see the directory `wuhan_data`.

- Move the data to the correct location. Assuming you are still in the same directory into which you cloned the `ml_exercise` repository, the command is `mv wuhan_data/top-tagging ml_exercise/data`

- In order to read this data, we need to install one final python package. Execute `pip install tables`.

You are now all set up for the exercise.

## 1. Top tagging using deep neural networks

We want to separate jets that originate from top quarks from jets that originate from gluons.

- **Task 1.1.** What is the difference between jets originating from top quarks and gluons?

<font color="blue">**Write down your solution here.**</font>

The separation of these jets is a difficult task and will be explained in the lecture. In this exercise we train a
*deep neural network* (DNN) for classifying jets. The DNN is trained with kinematic information of these jets. In
that way it learns to differentiate between jets as originating from a top quark or a gluon. This is a supervised
learning approach where we need labeled data to train the classifier.

In our case, we use jets produced with the Pythia8 parton shower for a center-of-mass energy of 14 TeV. The detector
response is simulated using Delphes simulating the ATLAS detector. The jets are clustered as fat jets using the
anti-$k_{\mathrm{T}}$ algorithm with a radius parameter of $R = 0.8$ (called *AK8-jets*). Jets are only considered in a
$p_{\mathrm{T}}$ range of $[550 - 650]\,\mathrm{GeV}$.

Due to the confinement of color charge at low energies, particles with color charge (i.e. gluons and quarks) form
bundles of color-neutral particles during the hadronization and parton shower processes after the hard scattering.
These color-neutral particles are referred to as jet constituents when clustered into jets. For each jet the
four-momenta of the first 200 jet constituents (sorted by momentum) are stored.

The training dataset is saved in a `HDF` file and loaded below. In this notebook the dataset is represented as `pandas`
data frame. The components of the four vectors are saved in columns with names `E_i`, `PX_i`, `PY_i`, `PZ_i` where `i`
is a number that runs from $0$ to $199$. The column `is_signal_new` is the label that tells us whether a jet originates
from a top quark decay (if `signal_new` has the value $1$) or from a gluon (if `signal_new` has the value `0`).

# 

In [ ]:
# first, make sure that all relevant packages are installed
# This should tell you that everything is already installed
!pip install tables

In [ ]:
%reset -f
from tensorflow import keras
import matplotlib.pyplot as plt
from plotfunctions import *
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,7)
mpl.rcParams['font.size'] = 14

# TODO adapt paths!
df_train = pd.read_hdf('../data/train.h5', 'table', stop=300000)
df_val = pd.read_hdf('../data/val.h5', 'table', stop=50000)
df_test = pd.read_hdf('../data/train.h5', 'table', start=300000, stop=350000)
df_secret = pd.read_hdf('../data/test_without_truth_100k.h5', 'table')

In [ ]:
df_train.head(5)

First we want to look at the distributions of some kinematic variables. All events in the training dataset
are labeled. Hence we can compare the distributions of top jets and gluon jets.

- **Task 1.2.** Plot the distributions for the transverse momentum $p_{\mathrm{T}}$ and the invariant mass
  $m_{\mathrm{inv}}$ of the jet, calculated with the 200 jet constituents with the highest $p_{\mathrm{T}}$. The code
  has already been implemented. Compare the distributions for jets originating from top quarks and for jet originating
  from gluons.

In [ ]:
def add_jet_pt_column(df):
    px = np.sum([df['PX_{0:d}'.format(i)] for i in range(200)], axis=0)
    py = np.sum([df['PY_{0:d}'.format(i)] for i in range(200)], axis=0)
    df['Pt'] = np.linalg.norm([px, py], axis=0)


def add_inv_mass_column(df):
    e = np.sum([df['E_{0:d}'.format(i)] for i in range(200)], axis=0)
    px = np.sum([df['PX_{0:d}'.format(i)] for i in range(200)], axis=0)
    py = np.sum([df['PY_{0:d}'.format(i)] for i in range(200)], axis=0)
    pz = np.sum([df['PZ_{0:d}'.format(i)] for i in range(200)], axis=0)
    df['MInv'] = np.sqrt(np.power(e, 2) - np.power(np.linalg.norm([px, py, pz], axis=0), 2))


# Add additional feature columns to the data frames here
for df in [df_train, df_val, df_test, df_secret]:
    add_jet_pt_column(df)
    add_inv_mass_column(df)

In [ ]:
fig, ax = plot_signal_background(df_train, 'Pt')
fig, ax = plot_signal_background(df_train, 'MInv')

<font color="blue">**Write down your solution here.**</font>

- **Task 1.3.** Look at the following distributions:

  - The four-vector components of the constituents with indices $0$, $10$, $30$ and $70$ (indicating that the
    jet constituent is that with the highest, tenth-highest, ... $p_{\mathrm{T}}$).
 
  - The transverse momentum of the constituents with indices $0$, $10$, $30$ and $70$.
  
  Also here, the code for adding the columns to the data frame and for plotting has already been implemented.
  What differences do you observe for the distribution of top jets and gluon jets?

In [ ]:
for i in [0, 10, 30, 70]:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10.5))
    fig, axes[0][0] = plot_signal_background(df_train, 'E_{}'.format(i), fig=fig, ax=axes[0][0])
    fig, axes[0][1] = plot_signal_background(df_train, 'PX_{}'.format(i), fig=fig, ax=axes[0][1])
    fig, axes[1][0] = plot_signal_background(df_train, 'PY_{}'.format(i), fig=fig, ax=axes[1][0])
    fig, axes[1][1] = plot_signal_background(df_train, 'PZ_{}'.format(i), fig=fig, ax=axes[1][1])

In [ ]:
def add_pt_constituent_column(df, i):
    df['Pt_{0:d}'.format(i)] = np.linalg.norm(np.array([df_train['PX_{0:d}'.format(i)].values, df_train['PY_{0:d}'.format(i)].values]), axis=0)
    
for i in range(20):
    add_pt_constituent_column(df_train, i)
add_pt_constituent_column(df_train, 30)
add_pt_constituent_column(df_train, 70)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10.5))
fig, axes[0][0] = plot_signal_background(df_train, 'Pt_0', fig=fig, ax=axes[0][0])
fig, axes[0][1] = plot_signal_background(df_train, 'Pt_10', fig=fig, ax=axes[0][1])
fig, axes[1][0] = plot_signal_background(df_train, 'Pt_30', fig=fig, ax=axes[1][0])
fig, axes[1][1] = plot_signal_background(df_train, 'Pt_70', fig=fig, ax=axes[1][1])

<font color="blue">**Write down your solution here.**</font>

A basic implementation for training a DNN has been prepared in the notebook.

- **Task 1.4.** Familiarize yourself with how the model is built and extract the following information from the code:

  - How many layers does this network have?
  
  - What is the reason for inserting the so-called dropout layers between the dense layers?
  
  - What activation functions are used?
  
  - What is the purpose of the loss function and which one is chosen for this task?
  
  - Where are the number of training epochs and the batch size defined? What is the meaning of these two parameters?
  

For answering the questions you can look at the official `keras` documentation pages of the used classes and methods:

- [keras.Sequential class](https://keras.io/api/models/sequential/)

- [keras.Input class](https://keras.io/api/layers/core_layers/input/)

- [keras.layers.Dense class](https://keras.io/api/layers/core_layers/dense/)

- [keras.layers.Dropout class](https://keras.io/api/layers/regularization_layers/dropout/)

- [keras.Model methods like compile and fit](https://keras.io/api/models/model_training_apis/)

In [ ]:
input_features = [c.format(i) for i in range(20) for c in ['E_{}', 'PX_{}', 'PY_{}', 'PZ_{}']]
data_train = df_train[input_features].values
labels_train = df_train['is_signal_new'].values
data_val = df_val[input_features].values
labels_val = df_val['is_signal_new'].values

model = keras.Sequential()

model.add(keras.Input(shape=(data_train.shape[1], )))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(name='loss'),
              metrics=['accuracy', ])

In [ ]:
batch_size = 10000
epochs = 10

model_history = model.fit(data_train, labels_train, validation_data=(data_val, labels_val), batch_size=batch_size, epochs=epochs)

<font color="blue">**Write down your solution here.**</font>

- **Task 1.5.** Now perform the training. After having finished the training, plot the loss and the accuracy history.

  - What do these curves tell you about the training?
  
  - How could over-training (e.g. over-fitting) be observed in these plots?

In [ ]:
fig, ax = plot_accuracy(model_history)
fig, ax = plot_loss(model_history)

<font color="blue">**Write down your solution here.**</font>

Finally we want to evaluate the separation power of the DNN we constructed above. For that we use a test dataset with
labeled jets. The network output value is evaluated for each jet in the dataset. Finally the distributions of the
network output value for jets that are labeled as top jets and for jets that are labeled as gluon jets.

An important measure for evaluate the separation power is the *ROC-AUC value* that is equal to the area under the
*ROC curve*. The ROC curve is calculated as follows: Several cut values for the network output are considered. For each
cut value events above that threshold are considered to be predicted as top quark jets and events below that threshold
are considered to be predicted as gluon jets. Following these assumptions the true positive rate
$$
  \mathrm{TPR} = \frac{\mathrm{top\,jets\,above\,cut}}{\mathrm{top\,jets}}
$$
and the false positive rate
$$
  \mathrm{FPR} = \frac{\mathrm{gluon\,jets\,above\,cut}}{\mathrm{gluon\,jets}}
$$
are calculated. Here, the true positive rate is called *signal efficiency*, while 1 minus the false positive rate is
callsed *background rejection*. For each cut value these two rates form a point of the ROC curve. The ROC-AUC value
is simply obtained by computing the area that is included by the ROC curve and the two axes of the coordinate system.

- **Task 1.6.** Look at the distributions of the network output value and the ROC curve. What do these curves tell you
  about the network's ability to separate jets originating from a top quark from jets originating from a gluon? How
  would you rate the discrimination power of the given network?

In [ ]:
data_test = df_test[input_features].values
labels_test = df_test['is_signal_new'].values
labels_predicted = model.predict(data_test, verbose=1)
fig, ax = plot_network_output(data_test, labels_test, labels_predicted)

In [ ]:
fig, ax = plot_roc_curve(labels_test, labels_predicted)

<font color="blue">**Write down your solution here.**</font>

- **Task 1.7.** Imagine how you can improve the network architecture in order to reach a higher ROC-AUC value. Test
  your ideas by modifying the present code in the notebook.

<font color="blue">**Write down your solution here.**</font>

In [ ]:
# Make predictions for DNN machine learning challenge
def predict_for_challenge(filename, df_secret, model, input_features):
    data_secret = df_secret[input_features].values
    labels_predicted = model.predict(data_secret, verbose=1)
    np.save(filename, labels_predicted)


# Change filename if necessary
predict_for_challenge('submission_predictions/group_xxxx_predictions_dnn_name.npy', df_secret, model, input_features)

## 2. Top tagging using convolutional neural networks

We will now try a different approach to tagging these top quarks, namely image recognition. The shape of a particle
physics detector like ATLAS or CMS is basically a cylinder. The surface of the cylinder can be unrolled along the
radial ($\phi$) and longitudinal ($\eta$) coordinates. This surface is a rectangle and can be divided into pixels.
Detected jet constituents can be filled into these pixels where the energies of the constituents can be transformed
into intensities.

For the purpose of this top tagging exercise jet images are provided which were already preprocessed to appear
homogeneous. How these images are processed will be described during the tutorial. (Note: A single jet covers only a
small part of the whole detector cylinder surface, hence needs to be cut and rotated appropriately to yield comparable
jet images.)

- **Task 2.1.** Plot some jet images with the provided code. This also produces an overlay of many jet images. What
  differences between top jet and gluon jet images do you observe? How can you explain these differences?

In [ ]:
%reset -f
from tensorflow import keras
import matplotlib.pyplot as plt
from plotfunctions import *
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,7)
mpl.rcParams['font.size'] = 14

# TODO adapt paths!
df_train = pd.read_hdf('../data/train_img.h5', 'table', stop=50000)
df_val = pd.read_hdf('../data/val_img.h5', 'table', stop=20000)
df_test = pd.read_hdf('../data/train_img.h5', 'table', start=100000, stop=120000)
df_secret = pd.read_hdf('../data/test_without_truth_img_100k.h5', 'table', stop=50000)

In [ ]:
df_train.head(5)

In [ ]:
fig, ax = plot_overlayed_jet_images(df_train, 'top')
fig, ax = plot_overlayed_jet_images(df_train, 'gluon')

<font color="blue">**Write down your solution here.**</font>

Image recognition works with a special type of neural networks, so-called *convolutional neural networks* (CNN). These
networks have special convolutional layers which essentially convolute the pixelated jet images with learned filters.
These filters can for example learn to detect edges, or sharpen or blur the images or even detect certain objects
within the full picture.

- **Task 2.2.** The notebook contains code that is needed to prepare the dataset for training and that defines a CNN
  architecture. What are the functions of the convolutional layer, the pooling layer and the flatten layer? Train th
  network and look at the evaluation plots.

  For answering the questions you can look at the official `keras` documentation pages of the used classes and methods:

  - [keras.layers.Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)

  - [keras.layers.MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/)

  - [keras.layers.Flatten](https://keras.io/api/layers/reshaping_layers/flatten/)

<font color="blue">**Write down your solution here.**</font>

In [ ]:
def reshape_for_training(df):
    pixels = ['img_{0:d}'.format(i) for i in range(1600)]
    data = np.expand_dims(np.expand_dims(df[pixels].values, axis=-1).reshape(-1, 40, 40), axis=-1)
    if 'is_signal_new' in df:
        labels = df['is_signal_new'].values
    else:
        labels = None
    return data, labels


data_train, labels_train = reshape_for_training(df_train)
data_val, labels_val = reshape_for_training(df_val)
data_test, labels_test = reshape_for_training(df_test)

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(data_train.shape[1], data_train.shape[2], data_train.shape[3])))
model.add(keras.layers.Conv2D(4, (4, 4), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(name='loss'),
              metrics=['accuracy', ])

In [ ]:
batch_size = 10000
epochs = 10

model_history = model.fit(data_train, labels_train, validation_data=(data_val, labels_val), batch_size=batch_size, epochs=epochs)

In [ ]:
labels_predicted = model.predict(data_test, verbose=1)

fig, ax = plot_accuracy(model_history)
fig, ax = plot_loss(model_history)
fig, ax = plot_roc_curve(labels_test, labels_predicted)
fig, ax = plot_network_output(data_test, labels_test, labels_predicted)

In [ ]:
# Make predictions for CNN machine learning challenge
def predict_for_cnn_challenge(filename, df_secret, model):
    data_secret, _ = reshape_for_training(df_secret)
    labels_predicted = model.predict(data_secret, verbose=1)
    np.save(filename, labels_predicted)


# Change filename if necessary
predict_for_cnn_challenge('submission_predictions/group_xxxx_predictions_cnn_name.npy', df_secret, model)

## 3. Challenge

Your task is to try to construct the best neural network architecture to achieve the best ROC-AUC values possible.
  
  - Prepare your best performing NNs (at most 3). Use the code cells at the end of the two exercises 1 and 2 that
    contain the functions `predict_for_cnn_challenge` or respectively `predict_for_dnn_challenge`. These functions
    take a model (and in the case of DNNs the list of input features), evaluate the predictions on a secret test
    dataset and save the predictions to a `npy` file in the folder `submission_predictions`. Insert a name for your
    group into the filename. If you train more than one network also insert a name. This filename will label your
    submitted predictions for the ranking.

  - Upload your `.npy` files with the predicted classes to moodle.
  
  - The predicted labels you submitted will be compared with the true labels of the jets.

  - The winner will be announced in class on January 10th and will be rewarded an epic prize.